# INTELIGENCIA ARTIFICIAL (INF 371)
Dr. Edwin Villanueva

#  Laboratorio 1:  Problema del Laberinto y agente solucionador

En este notebook se implementa un Agente Simple de Solución de Problemas en entornos de Laberinto. 

El laberinto es leido de un archivo de texto que puede terner los siguientes caracteres:

    '#': Indica que es una celda con obstaculo  (impasable) 
    '~': Indica que es una celda con agua (pasable, con costo 5)
    ' ': Indica que es una celda vacia (passable con costo 1)
    'S': Indica que es celda de Inicio
    'G': Indicates que es celda Objetivo (premio de 100)
    'I': Indicates que es celda objetivo Intermedio (premio de 100)
 


# Funciones para leer el laberinto de disco

<b>readMazeFromFile</b>  La funcion readMazeFromFile lee un archivo de disco que contiene un laberinto y retorna una matriz de celdas del laberinto leido (grid).    grid es una lista de listas, ejemplo: [['#','S',' '],['#',' ','G'],['~','#','~']]

In [35]:
def readMazeFromFile(file):
    fin = open(file)
    lines = fin.read().splitlines()
    grid = []
    for line in lines: 
        grid.append(list(line))
    return grid

<b>getLocCells:</b> La funcion getLocCells devuelve la localizacion de la celda inicial (S) y celda objetivo (G) en un grid

In [36]:
def getLocCells(grid):
    numRows = len(grid)
    numCols = len(grid[0])
    for i in range(numRows):
        for j in range(numCols):
            if len(grid[i]) != numCols:
                raise "Grid no Rectangular"
            if grid[i][j] == 'S':
                cell_S = (i,j)
            if grid[i][j] == 'G':
                cell_G = (i,j)
            if grid[i][j] == 'I':
                cell_I = (i,j)
                
    if cell_S == None:
        raise "No hay celda de Inicio"
    if cell_G == None:
        raise "No hay celda Objetivo (G)"
    if cell_I == None:
        raise "No hay celda de Objetivo Intermedio (I)"        
        
    return cell_S, cell_G, cell_I 

#### <b> Funciones para mostrar los resultados en el laberinto</b> 

In [37]:
def grid2Str(grid):
    """ Convierte un grid a un string para vizualizacion """
    numRows = len(grid)
    numCols = len(grid[0])
    strGrid = []
    headerLine = ' ' + ('-' * (numCols)) + ' '
    strGrid.append(headerLine)
    for row in grid:
        rowLine = '|' + ''.join(row) + '|'
        strGrid.append(rowLine)
    strGrid.append(headerLine)
    return '\n'.join(strGrid)

In [38]:
def showAgent(grid, agent_location, agent_performance):
    """ Muestra los resultados en el grid.   """
    grid_copy = []
    for row in grid:
        grid_copy.append([x for x in row]) 
    row,col = agent_location
    grid_copy[row][col] = 'X' 
    print (grid2Str(grid_copy))
    print ('Desempeño del agente = {}'.format(agent_performance))
    print ("-------------------------------")

In [39]:
def displaySearchResults(grid, visitedNodes, solutionNodes):
    """ Muestra los resultados de busqueda en el grid.   """
    grid_copy = []
    for row in grid:
        grid_copy.append([x for x in row]) 
    for node in visitedNodes:
        row,col = node.state
        ch = grid[row][col]
        if ch != 'S' and ch != 'G': grid_copy[row][col] = 'o' 
    for node in solutionNodes:  
        row,col = node.state
        ch = grid[row][col]
        if ch != 'S' and ch != 'G': grid_copy[row][col] = 'x'    
    print (grid2Str(grid_copy))
    print ("x - celdas en la solucion")
    print ("o - celdas visitadas durante la busqueda")
    print ("-------------------------------")

# Implementacion del Entorno  


#### Clase <b>Thing</b>

  Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>. (No editar)  

In [40]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

#### Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [41]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepcion que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una accion 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localizacion especificada."""
        raise NotImplementedError

    def is_done(self):
        """Retorna True si no hay ningun agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localizacion location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        
        if location is not None:
            thing.location = location 
        else:
            #print('colocara localizacion por defecto')
            self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

#### Clase <b>MazeEnvironment</b>

Esta clase implementa el entorno del laberinto. El estado percibido por un agente en este entorno corresponde a la tupla [grid, location]. Las acciones posibles para un agente son: 'N' (ir una celda al norte), 'S' (ir una celda al sur), 'W' (ir una celda al oeste), 'E' (ir una celda al este)  

In [42]:
class MazeEnvironment(Environment):
    
    def __init__(self, grid):
        super().__init__()
        self.grid = grid
        self.numRows = len(grid)
        self.numCols = len(grid[0])
        self.cell_S, self.cell_G, self.cell_I = getLocCells(grid) #obtiene localizaciones de celdas S, G, I
        self.flag_G = False  # Flag que indica que un agente visitó previamente a la celda G 
        self.flag_I = False  # Flag que indica que un agente visito previamente a la celda I 
        
    def default_location(self, agent):
        """Coloca Localización por defecto a un agente sin localizacion especificada (en celda 'S')."""
        agent.location = self.cell_S
        
    def thing_classes(self):
        return [Agent]

    def percept(self, agent):
        """Retorna el estado del ambiente (location)"""
        return agent.location
    
    def __isValidLocation(self,location):
        """ Retorna true si la localizacion dada corresponde a una celda no bloqueada valida """
        row,col = location
        if row < 0 or row >= self.numRows:
            return False
        if col < 0 or col >= self.numCols:
            return False
        return not self.grid[row][col] == '#'       

    def execute_action(self, agent, action):
        """Implementa el MAPA De TRANSICION: Cambia la posicion del agente de acuerdo a la accion solicitada """
        row,col = agent.location   # obtiene posicion actual del agente
        
        if action == 'N' and self.__isValidLocation((row-1, col)):
            agent.location = (row-1, col)
            agent.performance -= 1
        elif action == 'S' and self.__isValidLocation((row+1, col)):
            agent.location = (row+1, col)
            agent.performance -= 1
        elif action == 'W' and self.__isValidLocation((row, col-1)):
            agent.location = (row, col-1)
            agent.performance -= 1
        elif action == 'E' and self.__isValidLocation((row, col+1)):
            agent.location = (row, col+1)
            agent.performance -= 1
            
        if agent.location == self.cell_G and self.flag_G == False :
            agent.performance += 100     # suma 100 puntos al agente cuando llega por primera vez a celda G 
            self.flag_G == True
            
        if agent.location == self.cell_I and self.flag_I == False :
            agent.performance += 100     # suma 100 puntos al agente cuando llega por primera vez a celda I 
            self.flag_I == True

# Implementacion de un Agente Solucionador  para el Laberinto  


#### Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [43]:
import collections
import random

class Agent(Thing):
    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

#### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [44]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

####  <b> Clase MazeSearchProblem </b>  
Esta es una subclase de SearchProblem que implementa concretamente el problema de busqueda en laberinto. El constructor recibe el grid del laberinto, una localizacion inicial y una una localizacion objetivo. Cada estado es representado internamente como una tupla (row,col) que indica la posicion del agente en una celda del grid. 

In [45]:
class MazeSearchProblem(SearchProblem):
    def __init__(self, grid, initial, goal):
        """El constructor recibe el grid, localizacion inicial y localizacion objetivo"""
        self.grid = grid
        self.numRows = len(grid)
        self.numCols = len(grid[0])
        self.initial = initial
        self.goal = goal
        
        self.numNodesExpanded = 0        
        self.expandedNodeSet = {}   
        
    def __isValidState(self,state):
        """ Retorna true si el estado dado corresponde a una celda no bloqueada valida """
        row,col = state
        if row < 0 or row >= self.numRows:
            return False
        if col < 0 or col >= self.numCols:
            return False
        return not self.grid[row][col] == '#'         

    def actions(self, state):
        """Retorna las acciones legales desde la celda actual """
        row,col = state
        acciones = []
        if self.__isValidState((row-1, col)):
            acciones.append('N')
        if self.__isValidState((row+1, col)):
            acciones.append('S')
        if self.__isValidState((row, col-1)):
            acciones.append('W')            
        if self.__isValidState((row, col+1)):
            acciones.append('E')            
        return acciones
    
    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde la celda actual.
        La accion debe ser alguna de self.actions(state)"""  
        row,col = state
        newState = ()
        if action == 'N':
            newState = (row-1, col)
        if action == 'S':
            newState = (row+1, col)
        if action == 'W':
            newState = (row, col-1)    
        if action == 'E':
            newState = (row, col+1)          
        return newState
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion sale del grid """
        row, col = state2
        if self.grid[row][col] == ' ':  # si celda es vacia
            actionCost = 1
        elif self.grid[row][col] == '~': # si celda tiene agua
            actionCost = 5
        elif self.grid[row][col] == 'I' or self.grid[row][col] == 'G' or self.grid[row][col] == 'S':
            actionCost = 1
        else:
            raise "El costo de una celda bloqueada no esta definido" 
        return c + actionCost

#### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

In [46]:
class Node:

    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        return Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next_state))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

#### Clase <b>FIFOQueue</b> 
Define la clase que implementa una cola tipo FIFO First-In-First-Out (para BFS)

In [47]:
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

#### <b>Algoritmo general de búsqueda con memoria de estados visitados (Graph Search)</b>

Busca en los sucesores de un problema para encontrar un objetivo. El argumento frontier debe ser una cola vacia. Si dos caminos arrivan al mismo estado, solo usa el primero.

In [48]:
def graph_search(problem, frontier):

    frontier.append(Node(problem.initial))
    explored = set()
    visited_nodes = []   # almacena nodos visitados durante la busqueda (para fines de visualizacion)
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)   # nodo que se toma de la frontera es nodo visitado 
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### Programa de agente solucionador de problemas para el entorno MazeEnvironment


In [49]:
class MazeSearchAgentProgram:
    def __init__(self, grid, search_method):
        self.grid = grid
        self.initial, self.cell_G, self.cell_I = getLocCells(grid) # obtiene celdas S, G, I del laberinto
        self.goal = self.cell_G  # celda G  es el objetivo por defecto del agente
        self.search_method = search_method
        self.seq = []  # lista de acciones a ejecutar, inicialmente vacia
        
    def __call__(self, percept):
        state = percept
        print('Agente percibiendo localizacion = {}'.format(state))
        
        if state == self.goal:    # Si el ambiente esta en el estado objetivo no hace nada
            return 'None'
        if not self.seq:  # si la lista de acciones esta vacia
            print('Planeando solucion: estado_inicial = {}. estado_objetivo={}'.format(state,self.goal))
            search_problem = MazeSearchProblem(self.grid, state, self.goal) # instancia el problema de busqueda
            if self.search_method == 'bfs':
                # busca solucion con busqueda en amplitud
                goal_node, visited_nodes = graph_search(search_problem, FIFOQueue()) # frontera tipo cola FIFO 
            elif self.search_method == 'dfs':
                # busca solucion con busqueda en profundidad
                goal_node, visited_nodes = graph_search(search_problem, []) # frontera tipo pila ([] es una pila en Python)
            else:
                raise NotImplementedError
                        
            if goal_node == None: # sin solucion
                print('No se pudo encontrar solucion para el maze con metodo {}'.format(self.search_method) )
                return 'None'
            
            self.seq = goal_node.solution()
            print('Solution found with {}: Seq = {}. visited_nodes={}. path_cost = {}'.format(self.search_method, self.seq, len(visited_nodes),goal_node.path_cost))
            displaySearchResults(self.grid, visited_nodes, goal_node.path())
            
        action = self.seq.pop(0)       
        return action 

In [50]:
class MazeSearchAgentProgram_multigoal:
    def __init__(self, grid, search_method):
        self.grid = grid
        self.initial, self.cell_G, self.cell_I = getLocCells(grid) # obtiene celdas S, G, I del laberinto
        self.goal1 = self.cell_I # celda G  es el objetivo por defecto del agente
        self.goal2 = self.cell_G
        self.search_method = search_method
        self.seq = []  # lista de acciones a ejecutar, inicialmente vacia
        self.presentgoal = self.goal1
        self.total_cost = 0
        self.total_moves = 0
        
    def __call__(self, percept):
        state = percept
        print('Agente percibiendo localizacion = {}'.format(state))
        if state == self.presentgoal: # Si el ambiente esta en el estado objetivo no hace nada
            if state == self.goal1:
                self.presentgoal = self.goal2
                self.seq.clear()
            elif state == self.goal2:
                return 'None'
        if not self.seq:  # si la lista de acciones esta vacia
            print('Planeando solucion: estado_inicial = {}. estado_objetivo={}'.format(state,self.presentgoal))
            search_problem = MazeSearchProblem(self.grid, state, self.presentgoal) # instancia el problema de busqueda
            if self.search_method == 'bfs':
                # busca solucion con busqueda en amplitud
                goal_node, visited_nodes = graph_search(search_problem, FIFOQueue()) # frontera tipo cola FIFO 
            elif self.search_method == 'dfs':
                # busca solucion con busqueda en profundidad
                goal_node, visited_nodes = graph_search(search_problem, []) # frontera tipo pila ([] es una pila en Python)
            else:
                raise NotImplementedError
                        
            if goal_node == None: # sin solucion
                print('No se pudo encontrar solucion para el maze con metodo {}'.format(self.search_method) )
                return 'None'
            
            self.seq = goal_node.solution()
            self.total_cost = goal_node.path_cost + self.total_cost
            print('Solution found with {}: Seq = {}. visited_nodes={}. path_cost = {}'.format(self.search_method, self.seq, len(visited_nodes),goal_node.path_cost))
            displaySearchResults(self.grid, visited_nodes, goal_node.path())
         
            if self.presentgoal == self.goal2:
                self.total_moves = len(self.seq) + self.total_moves
                print('>>>>>>>>> Costo total = {}'.format(self.total_cost))
                print('>>>>>>>>> Movimientos totales = {}'.format(self.total_moves))
            else:
                self.total_moves = len(self.seq)
                
        action = self.seq.pop(0)       
        return action 

# Probando el agente solucionador de problemas en MazeEnvironment

In [51]:
# Carga un laberinto de archivo en disco, instancia el entorno y visualiza el grid
mazegrid = readMazeFromFile('maze3.txt') 
e = MazeEnvironment(mazegrid)
print(grid2Str(mazegrid))


 ------------------- 
|~~~~~~~~~~~~~~~~~~~|
|~~~~~~~~~~~~~~~~~~~|
|~~~~~          ~~~~|
|~~G~            ~~~|
|########   ~~~~~~~~|
|             ######|
|      S      #~~~~~|
|###### # #####~~~~~|
|     # #      ~~~~~|
| # # ###### ####~~~|
| #          #~~~~~~|
|     ### ####~~~~~~|
|                  I|
 ------------------- 


In [52]:
# Instancia el agente de solucion de problemas y lo añade al entorno del laberinto
a = Agent( MazeSearchAgentProgram_multigoal (mazegrid, 'bfs') ) 
e.add_thing(a) 

showAgent (mazegrid, a.location, a.performance)   # Agente es mostrado como una X


 ------------------- 
|~~~~~~~~~~~~~~~~~~~|
|~~~~~~~~~~~~~~~~~~~|
|~~~~~          ~~~~|
|~~G~            ~~~|
|########   ~~~~~~~~|
|             ######|
|      X      #~~~~~|
|###### # #####~~~~~|
|     # #      ~~~~~|
| # # ###### ####~~~|
| #          #~~~~~~|
|     ### ####~~~~~~|
|                  I|
 ------------------- 
Desempeño del agente = 0
-------------------------------


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [53]:
# Ejecuta 30 pasos del ambiente 
e.run(60)
showAgent (mazegrid, a.location, a.performance)  # Agente es mostrado como una X

Agente percibiendo localizacion = (6, 6)
Planeando solucion: estado_inicial = (6, 6). estado_objetivo=(12, 18)
Solution found with bfs: Seq = ['E', 'E', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'E', 'E']. visited_nodes=47171. path_cost = 42
 ------------------- 
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|ooGoooooooooooooooo|
|########ooooooooooo|
|ooooooooooooo######|
|ooooooSxxoooo#ooooo|
|######o#x#####ooooo|
|    o#o#xxxxxxxxxoo|
| # #o######o####xoo|
| #oooooooooo#oooxoo|
|   oo###o####~ooxoo|
|    ooooooooo ooxxx|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------
Agente percibiendo localizacion = (6, 7)
Agente percibiendo localizacion = (6, 8)
Agente percibiendo localizacion = (7, 8)
Agente percibiendo localizacion = (8, 8)
Agente percibiendo localizacion = (8, 9)
Agente percibiendo localizacion = (8, 10)
Agente percibiendo localizacion = (8, 11)
Agente perci

## Preguntas

<b>1) Para el problema del 'maze3.txt', indique los valores b, d, m (considere buqueda sin estados repetidos)</b>(3 puntos)
    
      - b: máximo factor de ramificación
      - d: profundidad del nodo objetivo menos profundo
      - m: profundidad máxima del arbol

<b>2) De acuerdo a los resultados encontrados con bfs y dfs, ¿Cuál método explora menos celdas para encontrar la solucion y cuál garantiza optimalidad? Explique relacionando con la teoría </b> (4 puntos)

<b>3) Cuál de los metodos de busqueda ciega estudiados en clase cree que sería más adecuado para el programa de agente solucionador de problemas de laberintos de cualquier tamaño? Justifique </b> (5 puntos)

<b>4)  Modifique al Programa del Agente (Cree una nueva funcion MazeSearchAgentProgram_multigoal) para que primero planifique y alcance la celda I, y luego desde allí, planifique y alcance la celda G. Escoja el algoritmo que encuentre soluciones con menos cantidad de movidas e indique el costo total de la solucion y numero de movidas desde celda S hasta G pasando por I</b> (8 puntos)

Pregunta 1:
       b: 4
       d: 12
       m: 96 

Pregunat2:
De acuerdo a laos resultados optenidos dfs explora muchos menos nodos que bfs. bfs exploro 1194 nodos y dfs exploro 59 nodos. Sin embargo, esto no significa que dfs siempre explore menos nodos, ya que esto depende del estado inicial.
Por otro lado ninguna de las dos formas de busqueda garantiza optimalidad, ya que dfs devuelve la primera solucion que encuentra, no necesariamnete la mas optima, y bfs solo es optimo si el costo de camino es el mismo en todos los casos.

Pregunta 3:
El mas adecuado seria el de busqueda de costo uniforme. Esto porque la bfs y dfs no garantizan ser optimas, la de profundidad limitada podria no encontrar una solucion si el objetivo esta por debajo de su limite. Ademas, la de profundidad iterativa y la de busqueda bidireccional son optimas solo cuando todas las acciones cuetan igual. Por lo tanto la unica opcion que nos asegura ser optima y encontrar la solucion es la de busqueda de costo uniforme. 

Pregunta 4:
El algoritmo que lo encuentra en menos movidas es bfs, sin embargo, es el que visita mas nodos

In [54]:
class MazeSearchAgentProgram_multigoal:
    def __init__(self, grid, search_method):
        self.grid = grid
        self.initial, self.cell_G, self.cell_I = getLocCells(grid) # obtiene celdas S, G, I del laberinto
        self.goal1 = self.cell_I # celda G  es el objetivo por defecto del agente
        self.goal2 = self.cell_G
        self.search_method = search_method
        self.seq = []  # lista de acciones a ejecutar, inicialmente vacia
        self.presentgoal = self.goal1
        self.total_cost = 0
        self.total_moves = 0
        
    def __call__(self, percept):
        state = percept
        print('Agente percibiendo localizacion = {}'.format(state))
        if state == self.presentgoal: # Si el ambiente esta en el estado objetivo no hace nada
            if state == self.goal1:
                self.presentgoal = self.goal2
                self.seq.clear()
            elif state == self.goal2:
                return 'None'
        if not self.seq:  # si la lista de acciones esta vacia
            print('Planeando solucion: estado_inicial = {}. estado_objetivo={}'.format(state,self.presentgoal))
            search_problem = MazeSearchProblem(self.grid, state, self.presentgoal) # instancia el problema de busqueda
            if self.search_method == 'bfs':
                # busca solucion con busqueda en amplitud
                goal_node, visited_nodes = graph_search(search_problem, FIFOQueue()) # frontera tipo cola FIFO 
            elif self.search_method == 'dfs':
                # busca solucion con busqueda en profundidad
                goal_node, visited_nodes = graph_search(search_problem, []) # frontera tipo pila ([] es una pila en Python)
            else:
                raise NotImplementedError
                        
            if goal_node == None: # sin solucion
                print('No se pudo encontrar solucion para el maze con metodo {}'.format(self.search_method) )
                return 'None'
            
            self.seq = goal_node.solution()
            self.total_cost = goal_node.path_cost + self.total_cost
            print('Solution found with {}: Seq = {}. visited_nodes={}. path_cost = {}'.format(self.search_method, self.seq, len(visited_nodes),goal_node.path_cost))
            displaySearchResults(self.grid, visited_nodes, goal_node.path())
         
            if self.presentgoal == self.goal2:
                self.total_moves = len(self.seq) + self.total_moves
                print('>>>>>>>>> Costo total = {}'.format(self.total_cost))
                print('>>>>>>>>> Movimientos totales = {}'.format(self.total_moves))
            else:
                self.total_moves = len(self.seq)
                
        action = self.seq.pop(0)       
        return action 